# Deploy on AWS Sagemaker

## Install the necessary packages

In [1]:
!pip install -U sagemaker==2.192.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 902.8/902.8 kB 17.6 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for sagemaker: filename=sagemaker-2.192.1-py2.py3-none-any.whl size=1206240 sha256=74298756815f019d2b956ac73d58a290b7c775239fe343583b53e59a3b2bc337
  Stored in directory: /home/ec2-user/.cache/pip/wheels/ba/74/23/fc5c71a7784765c4f3a262981272f28e4b5f812b74d20e1ebe
Successfully built sagemaker
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.188.0
    Uninstalling sagemaker-2.188.0:
      Successfully uninstalled sagemaker-2.188.0


## Deploy the Model as A SageMaker Endpoint

In [1]:
import sagemaker
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri
import time

sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name
role = sagemaker.get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
image_uri = get_huggingface_llm_image_uri(
  backend="huggingface", # or lmi
  region=region,
 version="1.1.0"
)

In [3]:
image_uri

'763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-tgi-inference:2.0.1-tgi1.1.0-gpu-py39-cu118-ubuntu20.04'

In [4]:
model_name = "MistralLite-" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

hub = {
    'HF_MODEL_ID':'amazon/MistralLite',
    'HF_TASK':'text-generation',
    'SM_NUM_GPUS':'1',
    "MAX_INPUT_LENGTH": '12000',
    "MAX_TOTAL_TOKENS": '12288',
    "MAX_BATCH_PREFILL_TOKENS": '12288',
    "MAX_BATCH_TOTAL_TOKENS":  '12288',
    "DTYPE": 'bfloat16',
}

model = HuggingFaceModel(
    name=model_name,
    env=hub,
    role=role,
    image_uri=image_uri
)
predictor = model.deploy(
  initial_instance_count=1,
  instance_type="ml.g5.2xlarge",
  endpoint_name=model_name,
    
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
----------------!

## Perform Inference

### Sagemaker SDK

In [5]:
input_data = {
  "inputs": "<|prompter|>What are the main challenges to support a long context for LLM?</s><|assistant|>",
  "parameters": {
    "do_sample": False,
    "max_new_tokens": 400,
    "return_full_text": False,
    #"typical_p": 0.2,
    #"temperature":None,
    #"truncate":None,
    #"seed": 1,
  }
}
result = predictor.predict(input_data)[0]["generated_text"]
print(result)

There are several challenges to supporting a long context for LLMs, including:

1. Memory Limitations: LLMs are limited in the amount of data they can store in memory, which can limit the length of the context they can support.

2. Computational Complexity: Generating a response for a long context can be computationally expensive, especially for larger models.

3. Training Data: Training an LLM to support a long context requires a large amount of training data, which can be difficult and time-consuming to collect.

4. Bias and Unanswerable Questions: As the context gets longer, there is a greater risk of bias and unanswerable questions, which can be difficult for the model to handle.

5. Human Evaluation: Evaluating the quality of responses for a long context can be challenging, as it may require human evaluators to read and understand a large amount of text.

6. Scalability: As the context gets longer, it becomes more difficult to scale the model to handle the increased computational 

### boto3

In [6]:
import boto3
import json
def call_endpoint(client, prompt, endpoint_name, paramters):
    client = boto3.client("sagemaker-runtime")
    payload = {"inputs": prompt,
               "parameters": parameters}
    response = client.invoke_endpoint(EndpointName=endpoint_name,
                                      Body=json.dumps(payload), 
                                      ContentType="application/json")
    output = json.loads(response["Body"].read().decode())
    result = output[0]["generated_text"]
    return result

client = boto3.client("sagemaker-runtime")
parameters = {
    "do_sample": False,
    "max_new_tokens": 400,
    "return_full_text": False,
    #"typical_p": 0.2,
    #"temperature":None,
    #"truncate":None,
    #"seed": 10,
}
endpoint_name = predictor.endpoint_name
prompt = "<|prompter|>What are the main challenges to support a long context for LLM?</s><|assistant|>"
result = call_endpoint(client, prompt, endpoint_name, parameters)
print(result)

There are several challenges to supporting a long context for LLMs, including:

1. Memory Limitations: LLMs are limited in the amount of data they can store in memory, which can limit the length of the context they can support.

2. Computational Complexity: Generating a response for a long context can be computationally expensive, especially for larger models.

3. Training Data: Training an LLM to support a long context requires a large amount of training data, which can be difficult and time-consuming to collect.

4. Bias and Unanswerable Questions: As the context gets longer, there is a greater risk of bias and unanswerable questions, which can be difficult for the model to handle.

5. Human Evaluation: Evaluating the quality of responses for a long context can be challenging, as it may require human evaluators to read and understand a large amount of text.

6. Scalability: As the context gets longer, it becomes more difficult to scale the model to handle the increased computational 